# Making Synthetic TI reports

The goal here is to fine-tune/train a language model on Threat Intelligence reports, and then use that model to generate new, fake, TI reports. I'm doing this mostly because I think it will be funny.

For this experiment, we're doing this with a very basic LSTM, rather than any of the fancier GPT or transformer models. (walk before you run and all that.)

In any case, the process here is:
 * collect a ton of TI reports. This is what the https://github.com/g-clef/ThreatIntelCollector is for.
 * parse them all to extract the text (that's what the prefect job in this repo is for)
 * make a fastai databundle out of them
 * instantiate a language learner
 * train it
 * predict text
 
This should be fairly straightforward, and I imagine this is going to be wonky at best. Ways to improve it that I can see right off the top:
 * replace the vendor URLs at the bottom of every page
 * parse out other URLs/hashes/DNS names/file paths in the report?
 * try gpt-2 (will it fit in my video card?)
 * try other fancier transformer libraries
 
For now, though, let's just do the default/easy one.

In [1]:
from fastai.text.all import *
from fastai import *

So, first I'm going to pull the data from the prefect job in (I copy the relevant files to a local drive on my ML rig so that I'm not paying a network round-trip price for every file), then make a DataLoader out of that data, setting aside 10% for a "valid" dataset. Note: this will also do some standard tokenization of the text (using Spacy, I believe)

In [11]:
base_dir = Path("/home/g-clef/local_ml_data_copy/ti-reports/bec011c7-0344-49dd-8daa-891f62a8c8f2")

In [19]:
dls_lm = TextDataLoaders.from_folder(base_dir, is_lm=True, valid_pct=0.1)

In [20]:
dls_lm.show_batch(max_n=5)

,text,text_
0,xxbos 12 / 5 / 2019 xxmaj obfuscation xxmaj tools xxmaj found in the xxmaj capesand xxmaj exploit xxmaj kit xxmaj possibly xxmaj used in “ kurdishcoder ” xxmaj campaign - trendlabs xxmaj security xxmaj intelligence xxmaj blog \n\n▁ xxmaj trend xxmaj micro \n▁ xxmaj about trendlabs xxmaj security xxmaj intelligence xxmaj blog \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n▁ xxmaj search : \n\n▁ xxmaj go to … \n\n▁ homecategories \n\n\n▁ xxmaj home » xxmaj exploits » xxmaj,12 / 5 / 2019 xxmaj obfuscation xxmaj tools xxmaj found in the xxmaj capesand xxmaj exploit xxmaj kit xxmaj possibly xxmaj used in “ kurdishcoder ” xxmaj campaign - trendlabs xxmaj security xxmaj intelligence xxmaj blog \n\n▁ xxmaj trend xxmaj micro \n▁ xxmaj about trendlabs xxmaj security xxmaj intelligence xxmaj blog \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n▁ xxmaj search : \n\n▁ xxmaj go to … \n\n▁ homecategories \n\n\n▁ xxmaj home » xxmaj exploits » xxmaj obfuscation
1,"all reported on widespread campaigns xxmaj education , government , financial services , energy , \n▁ similar to the activity we describe in this and the entertainment industries appear to be \n▁ report . xxmaj in xxmaj august 2014 xxmaj kaspersky described the most affected . xxmaj figure 5 depicts the share \n▁ the “ epic xxmaj turla ” xxunk while in xxmaj january of detection alerts for xxup witchcoven among \n▁","reported on widespread campaigns xxmaj education , government , financial services , energy , \n▁ similar to the activity we describe in this and the entertainment industries appear to be \n▁ report . xxmaj in xxmaj august 2014 xxmaj kaspersky described the most affected . xxmaj figure 5 depicts the share \n▁ the “ epic xxmaj turla ” xxunk while in xxmaj january of detection alerts for xxup witchcoven among \n▁ 2015"
2,"than 100 referenced sources for known as xxup apt28 ) , discovered [ 44 ] by xxup eset and the first xxup uefi rootkit found in the wild . \n\n▁ the xxmaj mobile matrix . \n▁ xxmaj in the xxmaj groups category , one of eset ’s contributions is xxmaj machete ( xxunk ) [ 45 ] , a xxunk \n\n▁ onage group with high - profile targets in xxmaj latin xxmaj","100 referenced sources for known as xxup apt28 ) , discovered [ 44 ] by xxup eset and the first xxup uefi rootkit found in the wild . \n\n▁ the xxmaj mobile matrix . \n▁ xxmaj in the xxmaj groups category , one of eset ’s contributions is xxmaj machete ( xxunk ) [ 45 ] , a xxunk \n\n▁ onage group with high - profile targets in xxmaj latin xxmaj american"
3,ngav and other endpoint in ltration \n▁ prevention solutions . \n\n▁ xxmaj mitigation \n\n\n\n▁ atombombing is performed just by using the underlying xxmaj windows mechanisms . xxmaj there is no need to exploit \n▁ operating system bugs or vulnerabilities . \n▁ http : / / blog.ensilo.com / atombombing - a - code - injection - that - bypasses - current - security - solutions 2 / 6 \n▁ 3 / 1 / 2017,and other endpoint in ltration \n▁ prevention solutions . \n\n▁ xxmaj mitigation \n\n\n\n▁ atombombing is performed just by using the underlying xxmaj windows mechanisms . xxmaj there is no need to exploit \n▁ operating system bugs or vulnerabilities . \n▁ http : / / blog.ensilo.com / atombombing - a - code - injection - that - bypasses - current - security - solutions 2 / 6 \n▁ 3 / 1 / 2017 atombombing
4,"spyware fork in its third version and \n\n▁ then to the complex spyware that is version 4 . xxmaj this last step is especially interesting , \n▁ showing a big leap from straightforward code functionality to highly sophisticated malware . \n\n▁ xxmaj this suggests the latest version may have been bought from vendors of specialist \n▁ surveillance tools . xxmaj that would n’t be surprising , as the market for these espionage","fork in its third version and \n\n▁ then to the complex spyware that is version 4 . xxmaj this last step is especially interesting , \n▁ showing a big leap from straightforward code functionality to highly sophisticated malware . \n\n▁ xxmaj this suggest

Next I'm going to make an NLP language learning model, in this case using a pre-trained LSTM model, and pointing that learning model at the dataloader from before.

In [22]:
learner = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=base_dir, wd=0.1).to_fp16()

By default, fastai language models are created in a "frozen" state, which means that only the head of the model will change with training, not the underlying layers. We'll see how well that works by just running a single epoch of training 

In [24]:
learner.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.565845,4.394007,0.287921,80.964211,21:44


In [25]:
learner.save("one_epoch")

Path('/home/g-clef/local_ml_data_copy/ti-reports/bec011c7-0344-49dd-8daa-891f62a8c8f2/models/one_epoch.pth')

To train the whole model (fine-tuning the already-trained model), I have to "unfreeze" the model, and train again. I don't know off the top of my head how long to train it for, but I'll take a guess at 10 epochs.

In [26]:
learner.unfreeze()
learner.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.956028,4.040853,0.326732,56.874836,22:26
1,3.593564,3.834614,0.345887,46.275551,22:10
2,3.460929,3.716889,0.357832,41.136227,22:33
3,3.234648,3.640720,0.366255,38.119286,21:48
4,3.179806,3.578173,0.373663,35.808056,21:39
5,3.142587,3.541600,0.376357,34.522125,21:38
6,3.009695,3.504174,0.381908,33.253979,21:48
7,3.001116,3.471208,0.387079,32.175575,21:32
8,3.037449,3.452090,0.389489,31.566305,21:42
9,2.904210,3.450108,0.390049,31.503803,21:31


It's looking like it's starting to level off here, so 10 epochs may be enough. Let's save it so I don't have to re-do that 200 minutes of training time, and then see how it does.

In [27]:
learner.save("eleven_epochs")
learner.save_encoder("first_finetuning")

To make predictions, we give the model a starting point (or starting words, in this case), and ask it to predict a number of words forward. Fortunately, lots of the models begin with a title, so we'll make one up. You can also add a "temperature" variable here, which is a way of (to my understanding) weighting the distribution of the words to be more or less "spiky". I'm not sure how that will effect the final results here, so I'm going to leave it alone for the first try.

In [28]:
first_try = learner.predict("Underpants Gnomes - A Targetted Intrusion", 2000)

In [29]:
print(first_try)

Xxunk Xxunk - a Targetted Intrusion on a Unioncryptoupdater`memory_exec2 
▁ Internetopena 


▁ Whatsapp 
▁ Pwn2own 

▁ tty - security 

▁ Posted on November 18 , 2012 .  Keep notified and encouraged to follow full categories 

▁ high-ﬁdelity 85321dee31100bd3ece5b586ac3e6557 부트 difﬁcult 

▁ Detecting Malware 


▁ Running tslow.pyc ! 

▁ Linux Backdoor [ mechanized ] 


▁ First Determines : False Positives 

▁ Successful Kill Chain : The RLO 


▁ Although the cyberthreat is considered in a just very segmented manner , which would better hearing the next attacks relevant 
▁ to APT , it may be even more difficult and thoroughly 1761 . Just like all the Linux threat 
▁ actors , this case envyscout Additionally , the overall form of obfuscation 

▁ with a slight bb28 splits out three odds of choice : it does n’t need to apply a different cybersecurity 

▁ capability when Sofacy works . We will expect these types of attacks to be conducted out of that 

▁ multiple kinds — among other governm

That is...bad. Almost funny, but still bad. A number of things strike me: 
 1. Why does each line start with `_`? I should probably cut that out of the original files if it's in there somewhere.
 2. The code samples/assembly language snipped that are scattered throughout are clearly confusing things. I should probably cut those out of the original files also.
 3. The rest of it isn't *terrible*, just nonsensicle. Curious to see how well it does with just the two fixes above before moving to other models (since I suspect those two fixes will make the other examples perform better also).

So, first step: let's find out where all those underscores are coming from.

Huh. Those underscores don't exist in the original files. It appears that is some sort of artifact of either Spacy or FastAI's tokenizer. I could try to change that behavior, but I may just leave it alone & remove them at the end. 

Next up: let's find the code examples and assembly and remove those. 

That will mean re-running the prefect side of this.

The results of that weren't perfect (`def` still shows up in a few places, for example), but it's much lower than it was, which I am hopeful will make the results easier to read, and less likely to spew out random assembly language instructions. So, let's start from the top with this new batch. 

In [2]:
base_dir = Path("/home/g-clef/local_ml_data_copy/ti-reports/1655302d-a401-4d87-a223-dbb06648bb8f")

In [3]:
dls_lm = TextDataLoaders.from_folder(base_dir, is_lm=True, valid_pct=0.1)

In [4]:
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxmaj search \n\n▁ xxmaj research xxmaj teaching xxmaj news xxmaj lab \n\n▁ xxmaj projects xxup gla2010 xxmaj in the xxmaj news xxmaj about \n▁ xxmaj publications xxmaj newsletter xxmaj people \n▁ xxmaj archives xxmaj events xxmaj opportunities \n▁ xxmaj contact \n\n\n\n\n▁ xxmaj between xxmaj hong xxmaj kong and xxmaj burma : xxmaj tracking xxup up007 and slserver \n▁ xxmaj espionage xxmaj campaigns \n\n▁ xxmaj april 18 , 2016 \n\n▁ xxmaj tagged","xxmaj search \n\n▁ xxmaj research xxmaj teaching xxmaj news xxmaj lab \n\n▁ xxmaj projects xxup gla2010 xxmaj in the xxmaj news xxmaj about \n▁ xxmaj publications xxmaj newsletter xxmaj people \n▁ xxmaj archives xxmaj events xxmaj opportunities \n▁ xxmaj contact \n\n\n\n\n▁ xxmaj between xxmaj hong xxmaj kong and xxmaj burma : xxmaj tracking xxup up007 and slserver \n▁ xxmaj espionage xxmaj campaigns \n\n▁ xxmaj april 18 , 2016 \n\n▁ xxmaj tagged :"
1,observed filename(s ) photo.exe \n▁ xxmaj related campaign - \n▁ xxmaj configured xxup c2 domain dns.undpus.com \n▁ xxmaj configured xxup c2 reporting gate http : / / dns.undpus.com / info.php \n▁ xxmaj configured 2nd stage payload http : / / dns.undpus.com / xxunk \n▁ xxup md5 xxunk \n▁ xxup sha256 xxunk \n\n\n▁ shimratreporter core \n▁ xxmaj observed filename(s ) loader.exe \n▁ xxmaj related campaign - \n▁ xxmaj configured xxup c2 domain dns.undpus.com,filename(s ) photo.exe \n▁ xxmaj related campaign - \n▁ xxmaj configured xxup c2 domain dns.undpus.com \n▁ xxmaj configured xxup c2 reporting gate http : / / dns.undpus.com / info.php \n▁ xxmaj configured 2nd stage payload http : / / dns.undpus.com / xxunk \n▁ xxup md5 xxunk \n▁ xxup sha256 xxunk \n\n\n▁ shimratreporter core \n▁ xxmaj observed filename(s ) loader.exe \n▁ xxmaj related campaign - \n▁ xxmaj configured xxup c2 domain dns.undpus.com \n▁
2,"hacker breaches xxmaj dutch security firm diginotar , allowing the xxmaj iranian \n▁ government to spy on xxmaj gmail users in xxmaj iran . xxmaj this remains one of the largest security \n▁ breaches in the history of the internet . \n\n\n▁ xxup april 2012 \n▁ xxmaj iranian oil infrastructure is targeted by sabotage malware agents xxmaj flame and xxmaj wiper . \n\n\n▁ xxup june 2012 \n▁ xxmaj new xxmaj york xxmaj","breaches xxmaj dutch security firm diginotar , allowing the xxmaj iranian \n▁ government to spy on xxmaj gmail users in xxmaj iran . xxmaj this remains one of the largest security \n▁ breaches in the history of the internet . \n\n\n▁ xxup april 2012 \n▁ xxmaj iranian oil infrastructure is targeted by sabotage malware agents xxmaj flame and xxmaj wiper . \n\n\n▁ xxup june 2012 \n▁ xxmaj new xxmaj york xxmaj times"
3,… … … … … … … … … … … … … . 11 \n▁ xxmaj installation … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … . 11 \n\n▁ xxmaj command and,… … … … … … … … … … … … . 11 \n▁ xxmaj installation … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … . 11 \n\n▁ xxmaj command and xxmaj
4,"most popular among threat actors , followed by trojanized office files ( doc ) \n▁ was the go - to distribution vector for other malware operators . xxmaj after the takedown , many xxunk \n▁ with 16 % . xxmaj in xxup t1 2021 , .net platform malware ( msil ) gained popularity and came in third with 9 % of \n▁ xxunk have had to rely on their own compromise vectors","popular among threat actors , followed by trojanized office files ( doc ) \n▁ was the go - to distribution vector for other malware operators . xxmaj after the takedown , many xxunk \n▁ with 16 % . xxmaj in xxup t1 2021 , .net platform malware ( msil ) gained popularity and came in third with 9 % of \n▁ xxunk have had to rely on their own compromise vectors or"


In [5]:
learner = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=base_dir, wd=0.1).to_fp16()

In [6]:
learner.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.671973,4.201200,0.314184,66.766373,19:16


In [7]:
learner.save("no_code_first_epoch")

Path('/home/g-clef/local_ml_data_copy/ti-reports/1655302d-a401-4d87-a223-dbb06648bb8f/models/no_code_first_epoch.pth')

In [8]:
learner.unfreeze()
learner.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.008717,3.824406,0.354689,45.805580,19:42
1,3.619409,3.626017,0.373919,37.562920,19:18
2,3.444314,3.516819,0.385200,33.677147,19:19
3,3.282677,3.456172,0.391376,31.695421,19:12
4,3.283823,3.377452,0.399238,29.296021,19:29
5,3.153641,3.335874,0.405628,28.102930,19:17
6,3.071716,3.295683,0.409922,26.995850,19:24
7,3.065157,3.265738,0.415158,26.199446,19:19
8,2.992585,3.249119,0.417202,25.767630,19:16
9,2.917643,3.245584,0.417723,25.676714,19:01


In [9]:
learner.save("no_code_eleven_epochs")
learner.save_encoder("no_code_first_finetuning")

In [10]:
second_try = learner.predict("Underpants Gnomes - A Targeted Intrusion", 2000)

In [11]:
print(second_try)

Xxunk Xxunk - a Targeted Intrusion with Network Uncovers Mirrorthief Group Targets Leveraging 

▁ VPN Domains 

▁ Meet 주소 , Tiered Architecture and foreign - policy Operations at Over 

▁ The Atlantic Treaty Organization 

▁ March 11 , 2018 at 10:21:44 PM 

▁ Vulnerability in Internet Explorer 10 

▁ 5638 - at - xpl0it - analysis 

▁ t+m - yoroi - blog 
▁ By Supplying 82053737 , a augustus 11.0 
▁ producer 

▁ October 2 , 2021 

▁ Evasive Tools of muddywater Unknown Unknown Threat Actor 

▁ Early 02 , 2020 






▁ Witnessed in the first wave of Ill - dated March 27 - 2020 attacks by espionage 
▁ in the Palestinian Authority . 


▁ Introduction 


▁ The Middle East contains an increase in cyber 
▁ capabilities , distributed denial - of - service ( dos attacks and the use of malicious 
▁ Ipsum Denial - of - service , Keybase Malware , spoofed emails and proper infection chain . 


▁ Copies of the malicious files ( jan . 13 , 2021 ) , ILLUSTRATING 

▁ darkhalo , Tortoiseshell ’s Playbook

This is a lot better. It's still bad, but not nearly as bad as before. The text is a little better, the IOC's are more sensical (though still wrong). It seemed to level off at about 41% accuracy, though, which isn't great. I'm happy with this, but I think this is as far as the LSTM will take me. Next stop: transformers.

Ok, I lied. One last thing: I want to check to see if lots more rounds of training will improve things, or if it's done (and/or overfit). Will just let this run in the background while I do other things.

In [7]:
learner = learner.load("no_code_eleven_epochs")

In [8]:
learner.unfreeze()


In [9]:
learner.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.084944,2.694850,0.484568,14.803301,21:00
1,3.073791,2.752350,0.476832,15.679433,20:26
2,3.085491,2.856236,0.462863,17.395916,20:36
3,3.016444,3.020363,0.439856,20.498728,20:25
4,3.075782,3.188663,0.417861,24.255976,20:31
5,3.072004,3.314531,0.406641,27.509478,20:26
6,3.138239,3.394563,0.398857,29.801619,20:26
7,3.122430,3.436461,0.397035,31.076775,19:54
8,2.995036,3.477359,0.395063,32.374096,20:04
9,3.037057,3.481994,0.393698,32.524525,19:56


welp, from an accuracy and perplexity point of view, that seems to have made things worse. Never mind.